In [1]:
import pandas as pd
from torch.utils.data import DataLoader, Dataset

/home/joao/virtualenvs/esquizo/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/joao/virtualenvs/esquizo/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
df = pd.read_csv('conversas_mexidas.csv',dtype=str)

In [3]:
print(df.columns)

Index(['Unnamed: 0', 'Conversa', 'Label'], dtype='object')


In [4]:
df.head()

,Unnamed: 0,Conversa,Label
0,0,podemos fazer isso rápido? roxanne korrine e a...,cmq
1,1,"bem, eu pensei que começaríamos com a pronúnci...",cma
2,2,não o hacking e engasgos e cuspir parte. por f...,cms
3,3,"ok, então que tal experimentarmos uma cozinha ...",csq
4,0,você está me convidando para sair. isso é tão ...,cmq


In [5]:
tst = df[df.Label != '0']

In [6]:
utterance = []
allo = []
tmpList = []
for index, row in tst.iterrows():
    allo.append(row['Conversa'])
    if row['Unnamed: 0'] == '0':
        if len(tmpList) != 0:
            utterance.append(tmpList)
            tmpList = []
    
    tmpList.append({'Conversa': row['Conversa'], 'Label': row['Label']})
    #tmpList.append([row['Conversa'], row['Label']])
        

In [7]:
chars = list({l for word in allo for l in word})

In [8]:
token_dict = {token:k for token, k in zip(chars, range(1,len(chars)+1))}

In [9]:
token_dict

{'?': 1,
 'í': 2,
 'm': 3,
 'f': 4,
 'd': 5,
 'q': 6,
 'o': 7,
 '3': 8,
 'a': 9,
 'c': 10,
 'ü': 11,
 'l': 12,
 'à': 13,
 'x': 14,
 '!': 15,
 'ã': 16,
 'w': 17,
 '-': 18,
 'õ': 19,
 'r': 20,
 'â': 21,
 'h': 22,
 'b': 23,
 'p': 24,
 'ê': 25,
 'á': 26,
 'v': 27,
 's': 28,
 ' ': 29,
 'ç': 30,
 'y': 31,
 ',': 32,
 '5': 33,
 't': 34,
 '4': 35,
 '.': 36,
 'g': 37,
 '0': 38,
 '7': 39,
 '…': 40,
 'u': 41,
 'é': 42,
 'ô': 43,
 'n': 44,
 '2': 45,
 'ú': 46,
 '"': 47,
 'j': 48,
 'z': 49,
 'e': 50,
 'k': 51,
 'i': 52,
 'ó': 53}

In [10]:
end_token = '<end>'
start_decoding_token = '<start>'
pad_token = '<pad>'


token_dict[pad_token] = 0
token_dict[end_token] = len(token_dict)
token_dict[start_decoding_token] = len(token_dict)

token_dict

{'?': 1,
 'í': 2,
 'm': 3,
 'f': 4,
 'd': 5,
 'q': 6,
 'o': 7,
 '3': 8,
 'a': 9,
 'c': 10,
 'ü': 11,
 'l': 12,
 'à': 13,
 'x': 14,
 '!': 15,
 'ã': 16,
 'w': 17,
 '-': 18,
 'õ': 19,
 'r': 20,
 'â': 21,
 'h': 22,
 'b': 23,
 'p': 24,
 'ê': 25,
 'á': 26,
 'v': 27,
 's': 28,
 ' ': 29,
 'ç': 30,
 'y': 31,
 ',': 32,
 '5': 33,
 't': 34,
 '4': 35,
 '.': 36,
 'g': 37,
 '0': 38,
 '7': 39,
 '…': 40,
 'u': 41,
 'é': 42,
 'ô': 43,
 'n': 44,
 '2': 45,
 'ú': 46,
 '"': 47,
 'j': 48,
 'z': 49,
 'e': 50,
 'k': 51,
 'i': 52,
 'ó': 53,
 '<pad>': 0,
 '<end>': 54,
 '<start>': 55}

In [11]:
decoder_dict = {token_dict[k]:k for k in token_dict.keys()}

decoder_dict

{1: '?',
 2: 'í',
 3: 'm',
 4: 'f',
 5: 'd',
 6: 'q',
 7: 'o',
 8: '3',
 9: 'a',
 10: 'c',
 11: 'ü',
 12: 'l',
 13: 'à',
 14: 'x',
 15: '!',
 16: 'ã',
 17: 'w',
 18: '-',
 19: 'õ',
 20: 'r',
 21: 'â',
 22: 'h',
 23: 'b',
 24: 'p',
 25: 'ê',
 26: 'á',
 27: 'v',
 28: 's',
 29: ' ',
 30: 'ç',
 31: 'y',
 32: ',',
 33: '5',
 34: 't',
 35: '4',
 36: '.',
 37: 'g',
 38: '0',
 39: '7',
 40: '…',
 41: 'u',
 42: 'é',
 43: 'ô',
 44: 'n',
 45: '2',
 46: 'ú',
 47: '"',
 48: 'j',
 49: 'z',
 50: 'e',
 51: 'k',
 52: 'i',
 53: 'ó',
 0: '<pad>',
 54: '<end>',
 55: '<start>'}

In [12]:
from torchnlp import text_encoders

In [13]:
#CRIA UM ENCODER BASEADO EM TODAS AS PALAVRAS DO CORPUS
encoder = text_encoders.StaticTokenizerEncoder(allo,tokenize = lambda s:s.split())

In [14]:
len(encoder.vocab)

2491

In [27]:
encoded = []
gen = (i for i in allo)
encoded.append(encoder.encode(gen.__next__()))

encoded

[tensor([ 5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22,
         23])]

In [30]:
encoded = [encoder.encode(linha) for linha in allo]

In [31]:
encoded

[tensor([ 5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22,
         23]),
 tensor([24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 29, 35]),
 tensor([36, 37, 38, 11, 39, 11, 40, 41, 42, 43]),
 tensor([44, 45, 27, 46, 47, 16, 48, 49, 50, 51]),
 tensor([52, 53, 54, 55, 56, 57,  7, 58, 59, 60, 61, 37, 62, 63, 64, 65]),
 tensor([66]),
 tensor([67, 36, 58, 68, 69, 70, 71, 36, 72, 16, 73, 74, 75]),
 tensor([76]),
 tensor([30, 77, 58, 78, 70, 79, 80, 81, 64, 16, 82, 83, 84, 64, 85, 68, 86, 25,
         36, 87, 88, 89, 27, 90, 91]),
 tensor([92, 27, 90, 93, 94, 95, 96, 37, 97, 98]),
 tensor([42, 99]),
 tensor([100,  36, 101,  90, 102, 103, 104, 105, 106, 107,  37, 108, 109,  45,
         110, 111,  32,  90, 112, 113, 114, 115, 116, 117]),
 tensor([  7,  58,  16, 118]),
 tensor([119,  32, 120, 121, 122, 123,  94, 124,  56, 125,  98]),
 tensor([126, 127,  37,  27,  87, 128]),
 tensor([ 25,  36, 129, 130, 111, 131, 132,  25, 129, 130, 133, 134, 111, 135,
          14, 136, 137, 

In [18]:
labels = []
for index, row in tst.iterrows():
    labels.append(row['Label'])

In [19]:
from torchnlp.utils import pad_tensor

In [28]:
padded = [pad_tensor(encoded[cvs], length=90) for cvs in range(len(encoded))]

In [29]:
padded

[tensor([ 5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22,
         23,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0])]